<a href="https://colab.research.google.com/github/amitchug/ALMlops/blob/main/Experimentation_Phase_2_with_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Programme in AI and MLOps
## A programme by IISc and TalentSprint
### Assignment: Research to Production Environment: Experimentation_Phase : 2. Pipeline_Building and Test case


## Learning Objectives


At the end of the experiment you will be able to:
* create custom classes required for processing  
* implement pipeline and train the model
* save the model



In [ ]:
# @title Download the datasets
from IPython import get_ipython

ipython = get_ipython()

notebook="U1_MH1_Data_Munging" #name of the notebook

def setup():
    from IPython.display import HTML, display
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/titanic.csv")
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/test_titanic.csv")
    print("Data downloaded successfully")
    return

setup()

In [ ]:
!ls

List of files present

### Setup Steps:

In [ ]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "" #@param {type:"string"}

In [ ]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "" #@param {type:"string"}

In [ ]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()

notebook= "Experimentation_Phase_2_with_test" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")

    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aimlops-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



## Import Required Packages

In [ ]:
# Importing required libraries

import re  # Regular expressions for text processing
import pandas as pd  # Data manipulation and analysis
import numpy as np  # Numerical computations

# Importing functions for model training and evaluation
from sklearn.model_selection import train_test_split  # Splitting dataset into training and testing sets
from sklearn.metrics import accuracy_score  # Evaluating model accuracy
from sklearn.preprocessing import StandardScaler  # Standardizing features by scaling

# Importing machine learning model
from sklearn.ensemble import RandomForestClassifier  # Random Forest classifier for classification tasks

# Importing visualization library
import matplotlib.pyplot as plt  # Plotting graphs and visualizations

# Importing joblib for saving and loading models
import joblib  # Helps in persisting trained models and scalers for future use

# Importing type hinting support for function annotations
from typing import List  # Used to specify lists in function type hints

# Importing pipeline utilities
from sklearn.pipeline import Pipeline  # Used to create machine learning pipelines

# Importing base classes for creating custom transformers
from sklearn.base import BaseEstimator, TransformerMixin  # Base classes to define custom preprocessing transformers

### **1. Pre-Pipeline-Steps: Load, Explore and Prepare the Data Set**

* Understand different features in the training dataset
* Understand the data types of each columns
* Notice the columns of missing values




In [ ]:
# Load the dataset from a CSV file into a pandas DataFrame
data = pd.read_csv("titanic.csv")

In [ ]:
data.head()

In [ ]:
# Getting information about the dataset
data.info()

## Data Cleaning and Processing


 ### 1.1 Working on "SibSp" & "Parch" columns:
Combine columns "SibSp" & "Parch" and create another column that represents the total passengers in one ticket with the name "family_size". In each ticket, there might be Siblings/Spouses (SibSp =Number of Siblings/Spouses Aboard) or Parents/Children (Parch=Number of Parents/Children Aboard ) along with the passenger who booked the ticket.

  

In [ ]:
def family_size(data_frame):
    """
    Compute the family size for each passenger in the Titanic dataset.

    Parameters:
    data_frame (pd.DataFrame): Input DataFrame containing 'SibSp' (siblings/spouses)
                               and 'Parch' (parents/children) columns.

    Returns:
    pd.DataFrame: A new DataFrame with an additional 'FamilySize' column.
    """
    # Create a copy of the original DataFrame to avoid modifying it directly
    df = data_frame.copy()

    # Calculate family size: Number of siblings/spouses + Number of parents/children + 1 (self)
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

    # Return the modified DataFrame with the new column
    return df


In [ ]:
# Apply the family_size function to the dataset
data = family_size(data)

# Display summary information about the DataFrame
data.info()

### 1.2 Working on "Cabin" column:
Find unique entries in the Cabin column. We can label all passengers in two categories having a cabin or not. Check the data type(use: type) of each entry of the Cabin. Convert a string data type into '1' i.e. passengers with cabin and others into '0' i.e. passengers without cabin.  Write a function for the above operation and apply it to the cabin column and create another column with the name " Has_cabin" containing only 0 or 1 entries.





In [ ]:
def process_cabin(data_frame):
    """
    Create a new feature indicating whether a passenger has a cabin or not.

    Parameters:
    data_frame (pd.DataFrame): Input DataFrame containing the 'Cabin' column.

    Returns:
    pd.DataFrame: A new DataFrame with an additional 'Has_cabin' column.
    """
    # Create a copy of the original DataFrame to avoid modifying it directly
    df = data_frame.copy()

    # Define a lambda function to check if a cabin entry exists
    # If 'Cabin' is a float (i.e., NaN), return 0 (No cabin), otherwise return 1 (Has cabin)
    f1 = lambda x: 0 if isinstance(x, float) else 1  # Ternary expression

    # Apply the lambda function to the 'Cabin' column to create the 'Has_cabin' feature
    df['Has_cabin'] = df['Cabin'].apply(f1)

    # Return the modified DataFrame
    return df

In [ ]:
# Apply the process_cabin function to the dataset
data = process_cabin(data)

# Display summary information about the DataFrame
data.info()

### 1.3  Working on "Name" column :
Fetch titles from the name. We can map these titles with numbers and convert them into an integer by mapping with relative numbers. Use: concept of the regular expression.

In [ ]:
# Function to extract the title (Mr, Mrs, Miss, Master, etc.) from the passenger's name
def get_title(passenger):
    """
    Extracts the title from a given passenger's name.

    Parameters:
    passenger (str): The full name of a Titanic passenger.

    Returns:
    str: Extracted title (e.g., 'Mr', 'Mrs', 'Miss', 'Master', or 'Other').
    """
    line = passenger  # Assigns the passenger's name to a variable

    # Check for common titles in the name and return the appropriate title
    if re.search('Mrs', line):  # Searches for 'Mrs' in the name
        return 'Mrs'
    elif re.search('Mr', line):  # Searches for 'Mr' in the name
        return 'Mr'
    elif re.search('Miss', line):  # Searches for 'Miss' in the name
        return 'Miss'
    elif re.search('Master', line):  # Searches for 'Master' in the name
        return 'Master'
    else:
        return 'Other'  # Returns 'Other' for uncommon or missing titles

In [ ]:
#Example:
print(get_title('Heranld Mr.'))
data['Name'].apply(get_title)

In [ ]:
# Apply the get_title function to the 'Name' column to extract titles (Mr, Mrs, Miss, etc.)
data['Title'] = data['Name'].apply(get_title)

# Explanation:
# 1. 'data["Name"]' selects the 'Name' column from the dataset.
# 2. '.apply(get_title)' applies the get_title function to each value in the 'Name' column.
# 3. The get_title function extracts and returns the title (e.g., 'Mr', 'Mrs', 'Miss', etc.).
# 4. The extracted title is stored in a new column named 'Title' in the DataFrame.

In [ ]:
# Print the unique values in the 'Title' column to see all distinct titles
print(data['Title'].unique())

# Get a summary of the DataFrame, including column names, data types, and non-null counts
data.info()

In [ ]:
# Drop unnecessary columns from the dataset
data.drop(labels=['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)

# Explanation:
# 1. 'data.drop()' is used to remove specific columns from the DataFrame.
# 2. 'labels=['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin']' specifies the list of column names to drop.
# 3. 'axis=1' indicates that columns (not rows) are being dropped.
# 4. 'inplace=True' ensures that the changes are applied directly to the DataFrame, instead of creating a new one.

# Display the summary of the DataFrame to check the remaining columns
data.info()

### **2. Pipeline-Steps**
### Building custom class compatible with sklearn pipeline for imputation, feature mapping and any specific operation on any column.

### **A. Imputation**

Buiding custom Imputation class compatible with Sklearn for 'Embarked' colum imputation.

In [ ]:
class embarkImputer(BaseEstimator, TransformerMixin):
    """embarked column imputer."""

    def __init__(self, variables: str):
        """
        Initialize the embarkImputer class.

        Parameters:
        variables (str): The name of the column to be imputed (e.g., 'Embarked').

        Raises:
        ValueError: If 'variables' is not a string.
        """
        # Ensure that 'variables' is a string (column name)
        if not isinstance(variables, str):
            raise ValueError("variables should be a list")  # Raise error if not a string

        self.variables = variables  # Assign the column name to self.variables

    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        """
        Fit the imputer by calculating the most frequent value (mode) of the column.

        Parameters:
        X (pd.DataFrame): The input data to compute the mode value from.
        y (pd.Series, optional): The target variable, not needed for this transformer.

        Returns:
        self: The fitted transformer object.
        """
        # Calculate the most frequent value (mode) of the specified column in the dataset
        self.fill_value = X[self.variables].mode()[0]
        return self  # Return the transformer instance (needed for compatibility with sklearn pipeline)

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Apply the transformation (fill missing values with the mode value).

        Parameters:
        X (pd.DataFrame): The input data with missing values in the specified column.

        Returns:
        pd.DataFrame: A DataFrame with missing values filled.
        """
        # Create a copy of the input data to avoid modifying the original DataFrame
        X = X.copy()

        # Fill the missing values in the specified column with the computed fill value
        X[self.variables] = X[self.variables].fillna(self.fill_value)

        return X  # Return the transformed DataFrame

In [ ]:
# Check for the number of missing (null) values in the 'Embarked' column
data.Embarked.isnull().sum()

# Explanation:
# 1. 'data.Embarked' refers to the 'Embarked' column in the DataFrame 'data'.
# 2. '.isnull()' generates a boolean Series where 'True' represents missing (NaN) values in the 'Embarked' column,
#    and 'False' represents non-missing values.
# 3. '.sum()' counts the number of 'True' values in the boolean Series, which corresponds to the number of missing values.
#    It adds up all the 'True' values (which are treated as 1) to return the total number of missing values in the column.

In [ ]:
# Create an instance of the embarkImputer class for the 'Embarked' column
emb = embarkImputer('Embarked')

# Fit the imputer on the 'data' DataFrame (calculates the most frequent value for 'Embarked')
emb.fit(data)

# Apply the imputer to transform the 'data' DataFrame (fills missing values in the 'Embarked' column)
print(len(emb.transform(data).Embarked))  # Print the length of the 'Embarked' column after transformation

# Save the transformed data in a new variable
data1 = emb.transform(data)

# Print the length of the 'Embarked' column in the transformed DataFrame
print(len(data1.Embarked))

# Check and print the number of missing (null) values in the 'Embarked' column of the transformed DataFrame
print(data1.Embarked.isnull().sum())  # This should print 0 if all missing values were filled

In [ ]:
data1.Embarked.dtypes

In [ ]:
data1.info()

### **B. Mapping**

Bulding Mapper class for mpping 'Embarked','Sex' and 'Title' columns

In [ ]:
class Mapper(BaseEstimator, TransformerMixin):
    """Categorical variable mapper."""

    def __init__(self, variables: str, mappings: dict):
        """
        Initialize the Mapper class with the specified column and mappings.

        Parameters:
        variables (str): The name of the categorical variable (column) to be mapped.
        mappings (dict): A dictionary where keys are the unique categorical values
                         in the column, and values are the new values to map them to.

        Raises:
        ValueError: If 'variables' is not a string.
        """
        # Ensure that 'variables' is a string (column name)
        if not isinstance(variables, str):
            raise ValueError("variables should be a str")  # Raise error if not a string

        self.variables = variables  # Assign the column name to self.variables
        self.mappings = mappings  # Assign the mapping dictionary to self.mappings

    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        """
        Fit method (no operation needed in this case, required for compatibility with the sklearn pipeline).

        Parameters:
        X (pd.DataFrame): The input data.
        y (pd.Series, optional): The target variable, not needed for this transformer.

        Returns:
        self: The fitted transformer object.
        """
        # Fit is not used for the mapper as the mapping is predefined
        return self  # Return the transformer instance (needed for compatibility with sklearn pipeline)

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        """
        Apply the transformation (map the categorical values in the column to the new values).

        Parameters:
        X (pd.DataFrame): The input data with categorical variables to be mapped.

        Returns:
        pd.DataFrame: A DataFrame with mapped categorical values in the specified column.
        """
        # Create a copy of the input data to avoid modifying the original DataFrame
        X = X.copy()

        # Map the categorical values in the specified column using the provided mappings dictionary
        # The 'map' function replaces each value in the column with its corresponding value from the mappings
        # After mapping, the column is converted to an integer type using 'astype(int)'
        X[self.variables] = X[self.variables].map(self.mappings).astype(int)

        return X  # Return the transformed DataFrame with mapped values

In [ ]:
# Create a Mapper instance to map the 'Sex' column: 'female' to 0 and 'male' to 1
map_sex = Mapper('Sex', {'female': 0, 'male': 1})

# Create a Mapper instance to map the 'Embarked' column:
# 'S' to 0, 'C' to 1, and 'Q' to 2
map_embarked = Mapper('Embarked', {'S': 0, 'C': 1, 'Q': 2})

# Create a Mapper instance to map the 'Title' column:
# 'Mrs' to 4, 'Master' to 3, 'Miss' to 2, 'Mr' to 1, and 'Other' to 0
map_title = Mapper('Title', {'Mrs': 4, 'Master': 3, 'Miss': 2, 'Mr': 1, 'Other': 0})

In [ ]:
data1.head()

In [ ]:
# Loop through the list of mappers (map_sex, map_embarked, map_title)
for i in [map_sex, map_embarked, map_title]:
    # Fit the current mapper on the 'data' DataFrame and transform it
    # The result is assigned back to 'data1' for each mapper
    data1 = i.fit(data).transform(data1)

In [ ]:
data1

In [ ]:
data.info()

### **C. Class for Specific operation : Age column transformation**

Creating Class for Age column transformation that is compatible with SK_learn pipeline:

In the pre-processing steps , a function was created for processing age column. Now we are converting that function into a class suitable for inserting inside the pipeline.

In [ ]:
# Importing necessary modules for creating a custom transformer
#from sklearn.base import BaseEstimator, TransformerMixin

class age_col_tfr(BaseEstimator, TransformerMixin):
    """Age column transformer"""

    def __init__(self, variables):
        """
        Initialize the transformer with the specified column to be transformed.

        Parameters:
        variables (str): The name of the column to transform, typically 'Age'.

        Raises:
        ValueError: If 'variables' is not a string.
        """
        # Ensure that 'variables' is a string (column name)
        if not isinstance(variables, str):
            raise ValueError('variables should be a str')  # Raise error if not a string
        self.variables = variables  # Assign the column name to self.variables

    def fit(self, X: pd.DataFrame, y=None):
        """
        Fit method (used to calculate the mean and standard deviation of the column to transform).

        Parameters:
        X (pd.DataFrame): The input data.
        y (optional): The target variable, not needed for this transformer.

        Returns:
        self: The fitted transformer object.
        """
        # Calculate the mean and standard deviation of the 'Age' column
        self.age_avg = X[self.variables].mean()  # Mean of the 'Age' column
        self.age_std = X[self.variables].std()   # Standard deviation of the 'Age' column

        # We need the fit method to make the transformer compatible with sklearn's pipeline
        return self  # Return the transformer instance after fitting

    def transform(self, X):
        """
        Transform method (used to fill missing values in the 'Age' column with random values).

        Parameters:
        X (pd.DataFrame): The input data to be transformed.

        Returns:
        pd.DataFrame: A DataFrame with missing values in the 'Age' column filled with random values
        drawn from a normal distribution.
        """
        np.random.seed(42)  # Set the seed for reproducibility of random values
        X = X.copy()  # Create a copy of the input data to avoid altering the original DataFrame

        # Count the number of missing values in the 'Age' column
        age_null_count = X[self.variables].isnull().sum()

        # Generate random values for missing 'Age' values, drawn from a normal distribution
        # The random values will be within one standard deviation of the mean
        age_null_random_list = np.random.randint(
            self.age_avg - self.age_std, self.age_avg + self.age_std, size=age_null_count
        )

        # Fill the missing values (NaN) in the 'Age' column with the generated random values
        X.loc[np.isnan(X[self.variables]), self.variables] = age_null_random_list

        # Convert the 'Age' column to integer type (since the random values are generated as integers)
        X[self.variables] = X[self.variables].astype(int)

        return X  # Return the transformed DataFrame with filled 'Age' values

In [ ]:
age_tfr=age_col_tfr('Age')

In [ ]:
age_tfr.fit(data1)

In [ ]:
data1=age_tfr.transform(data1)

In [ ]:
sum(data1.Age.isnull())

### **3. Building Pipeline**

Finally building pipeline and implementing all above class inside pipeline along with classifier also inside.

In [ ]:
# Create a pipeline named 'titanic_pipe' to process and model the Titanic dataset
titanic_pipe = Pipeline([

    # Embarked column imputation: fill missing values in the 'Embarked' column using the custom 'embarkImputer' class
    ('embark_imputation', embarkImputer(variables='Embarked')),

    ##========== Mapper: Transform categorical variables to numeric values ==========##

    # Map 'Sex' column: Convert 'female' to 0 and 'male' to 1
    ('map_sex', Mapper('Sex', {'female': 0, 'male': 1})),

    # Map 'Embarked' column: Convert 'S' to 0, 'C' to 1, and 'Q' to 2
    ('map_embarked', Mapper('Embarked', {'S': 0, 'C': 1, 'Q': 2})),

    # Map 'Title' column: Convert 'Mrs' to 4, 'Master' to 3, 'Miss' to 2, 'Mr' to 1, and 'Other' to 0
    ('map_title', Mapper('Title', {'Mrs': 4, 'Master': 3, 'Miss': 2, 'Mr': 1, 'Other': 0})),

    # Transform 'Age' column: Fill missing values in the 'Age' column with random values based on the mean and standard deviation
    ('age_transform', age_col_tfr(variables='Age')),

    # Scale the data: Standardize the features by scaling them to have a mean of 0 and a standard deviation of 1
    ('scaler', StandardScaler()),

    # Model: Fit a Random Forest Classifier with specified parameters
    ('model_rf', RandomForestClassifier(n_estimators=150, max_depth=5, random_state=42))
])

### Train_test_split

In [ ]:
# x contains all the features except the target variable 'Survived'
x = data.drop('Survived', axis=1)

# y contains the target variable 'Survived' (survival status)
y = data['Survived']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)

# Display the shapes of the training and testing sets
X_train.shape, X_test.shape

## **Writing Test cases for checking age_col_tfr**
This test sample is kept inside test_features.py. in  production code for testing purpose.

#### Try to understand missing index in age column in test set

In [ ]:
X_test.shape

In [ ]:
# Identify the indices of rows in X_test where the Age column has missing values (NaN)
X_test.loc[X_test.Age.isnull(), 'Age'].index

In [ ]:
np.isnan(X_test.loc[709,'Age'])

Using age_tfr class to fill the missing entires

In [ ]:
age_tfr = age_col_tfr('Age')

In [ ]:
#age_tfr.fit(X_train)
age_tfr.fit(X_test)

In [ ]:
age_column_filled = age_tfr.transform(X_test)
age_column_filled.loc[709,'Age']

### Training the model using pipeline above

In [ ]:
# Fit the pipeline on the training data (X_train and y_train)
titanic_pipe.fit(X_train, y_train)

# Use the fitted pipeline to make predictions on the test data (X_test)
y_pred = titanic_pipe.predict(X_test)

# Calculate and print the accuracy of the model on the test data
print("Accuracy(in %):", accuracy_score(y_test, y_pred) * 100)

### Persist the model

In [ ]:
# Save the trained pipeline (titanic_pipe) to a file using joblib
joblib.dump(titanic_pipe, 'titanic_pip.joblib')

### Checking for the versions may be used for requirements.txt file

In [ ]:
# Importing necessary libraries

import numpy as np  # For numerical operations and handling arrays
import pandas as pd  # For data manipulation and analysis
import sklearn  # For machine learning tasks (includes preprocessing, modeling, etc.)
import pydantic  # For data validation and settings management
import joblib  # For saving and loading models efficiently

In [ ]:
pip install pydantic

In [ ]:
pip install strictyaml

In [ ]:
pip install ruamel.yaml

In [ ]:
import strictyaml

In [ ]:
import ruamel.yaml

In [ ]:
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)
print(pydantic.__version__)
print(strictyaml.__version__)
print(ruamel.yaml.__version__)
print(joblib.__version__)

### Please answer the questions below to complete the experiment:




In [ ]:
#@title What is testing? {run: "auto", form-width: "500px", display-mode: "form" }
Answer = " " #@param [" ", "The process of creating software from scratch", "Identification and fixing software bugs","The practice of ensuring software meets specified requirements and functions as expected"]

In [ ]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [ ]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "" #@param {type:"string"}


In [ ]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "" #@param ["","Yes", "No"]


In [ ]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [ ]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")